# 🎤 Vietnamese ASR Training - Google Colab

**Nhận dạng giọng nói tiếng Việt với Wav2Vec2**

---

## 📋 Setup Checklist
- [ ] Runtime → Change runtime type → **GPU (T4)**
- [ ] Mount Google Drive
- [ ] Upload dataset lên Drive
- [ ] Run all cells

---

## 1️⃣ Check GPU & Environment

In [ ]:
import torch
import sys

print("="*60)
print("🔧 Environment Info")
print("="*60)
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n✅ GPU Ready!")
else:
    print("\n⚠️ WARNING: GPU not available!")
    print("Go to: Runtime → Change runtime type → Hardware accelerator → GPU")

## 2️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Tạo working directory
DRIVE_ROOT = "/content/drive/MyDrive/VietnameseASR"
os.makedirs(DRIVE_ROOT, exist_ok=True)

print(f"\n✓ Drive mounted at: {DRIVE_ROOT}")
print("\n📂 Cấu trúc thư mục khuyến nghị:")
print(f"{DRIVE_ROOT}/")
print("  ├── data/               # Dataset files")
print("  │   ├── train.jsonl")
print("  │   ├── validation.jsonl")
print("  │   └── test.jsonl")
print("  ├── models/             # Checkpoints (auto-created)")
print("  └── final_model/        # Final output (auto-created)")

## 3️⃣ Install Dependencies

In [ ]:
%%capture
# Install packages (silent mode)
!pip install -q transformers datasets evaluate jiwer soundfile librosa accelerate tensorboard

In [ ]:
# Verify installation
import transformers
import datasets
import evaluate

print("✅ All packages installed successfully!")
print(f"   - transformers: {transformers.__version__}")
print(f"   - datasets: {datasets.__version__}")
print(f"   - evaluate: {evaluate.__version__}")

## 4️⃣ Clone Source Code từ GitHub

In [ ]:
import os
import sys

# Check if repo already exists
if os.path.exists('/content/Vietnamese-asr'):
    print("⚠️ Repository already exists, updating...")
    os.chdir('/content/Vietnamese-asr')
    !git pull origin main
else:
    print("📥 Cloning repository...")
    !git clone https://github.com/CheeseThuong/Vietnamese-asr.git
    os.chdir('/content/Vietnamese-asr')

# CRITICAL: Add to Python path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    
print(f"\n✅ Repository ready")
print(f"📂 Location: {os.getcwd()}")
print(f"🐍 Python path: {sys.path[0]}")

# Verify structure
print(f"\n📁 Repository structure:")
!ls -la
print(f"\n📁 src/ contents:")
!ls -la src/

In [ ]:
# Verify Python imports (DEBUG)
print("="*60)
print("🔍 Verifying imports...")
print("="*60)

try:
    from src.data.preprocessing import load_and_prepare_datasets
    from src.training.train_wav2vec2 import create_model, train_model
    print("✅ All imports successful!")
except ModuleNotFoundError as e:
    print(f"❌ Import failed: {e}")
    print("\n🔧 Troubleshooting:")
    print(f"  Current dir: {os.getcwd()}")
    print(f"  Python path: {sys.path[0]}")
    print(f"  src/ exists: {os.path.exists('src')}")
    
print("="*60)

## 5️⃣ Check Dataset

**Hướng dẫn upload dataset:**
1. **Trên máy local**, chạy: `python convert_to_relative_paths.py`
2. Upload 3 files từ `processed_data_vivos/` lên Google Drive
3. Đặt vào: `MyDrive/VietnameseASR/data/`

**⚠️ QUAN TRỌNG**: Files phải có **relative paths** (ví dụ: `Data/vivos/vivos/train/...`) chứ KHÔNG phải absolute paths (`D:\Projects\...`)

In [ ]:
from pathlib import Path
import json

# Đường dẫn dataset
DATA_DIR = Path(f"{DRIVE_ROOT}/data")

# Kiểm tra files
required_files = ['train.jsonl', 'validation.jsonl', 'test.jsonl']
missing = [f for f in required_files if not (DATA_DIR / f).exists()]

if missing:
    print("❌ Missing dataset files:")
    for f in missing:
        print(f"   - {f}")
    print(f"\n📁 Expected location: {DATA_DIR}")
    print("\n💡 Upload dataset files to Google Drive first!")
else:
    print("✅ All dataset files found!")
    # Count samples and check paths
    for file in required_files:
        filepath = DATA_DIR / file
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            count = len(lines)
            
        # Check sample path
        if lines:
            sample = json.loads(lines[0])
            audio_path = sample['audio_path']
            is_relative = not os.path.isabs(audio_path)
            path_type = "✅ relative" if is_relative else "⚠️ absolute"
            
            print(f"   - {file}: {count:,} samples ({path_type})")
            print(f"     Sample path: {audio_path[:80]}...")
            
            # Warning if absolute paths found
            if not is_relative:
                print(f"     ⚠️ WARNING: Absolute paths detected! Will fail on Colab.")
                print(f"     💡 Run 'python convert_to_relative_paths.py' locally first!")
        else:
            print(f"   - {file}: {count:,} samples")

In [ ]:
# Copy dataset files từ Drive vào working directory của Colab
import shutil

print("📋 Setting up dataset in Colab workspace...")

# Tạo folder Data trong Colab
COLAB_DATA_DIR = Path("/content/Vietnamese-asr/Data")
COLAB_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Copy folder vivos từ Drive (nếu có)
DRIVE_VIVOS = Path(f"{DRIVE_ROOT}/vivos")
COLAB_VIVOS = COLAB_DATA_DIR / "vivos"

if DRIVE_VIVOS.exists():
    print(f"📂 Copying VIVOS dataset from Drive...")
    print(f"   Source: {DRIVE_VIVOS}")
    print(f"   Destination: {COLAB_VIVOS}")
    
    if COLAB_VIVOS.exists():
        shutil.rmtree(COLAB_VIVOS)
    shutil.copytree(DRIVE_VIVOS, COLAB_VIVOS)
    print(f"   ✅ Copied successfully!")
else:
    print(f"⚠️ VIVOS audio folder not found at: {DRIVE_VIVOS}")
    print(f"💡 Make sure to upload the 'vivos' folder to Drive")
    print(f"   Expected structure: {DRIVE_ROOT}/vivos/vivos/train/waves/...")

# Verify
if COLAB_VIVOS.exists():
    # Count audio files
    wav_files = list(COLAB_VIVOS.rglob("*.wav"))
    print(f"\n✅ Audio files ready: {len(wav_files):,} WAV files")
else:
    print(f"\n❌ Audio files not found!")

## 6️⃣ Training Configuration

In [ ]:
import json

# Configuration - Tối ưu cho Colab GPU
config = {
    'pretrained_model': 'nguyenvulebinh/wav2vec2-base-vietnamese-250h',
    'num_train_epochs': 30,          # Số epochs
    'batch_size': 16,                # GPU T4 ~ 16GB RAM
    'gradient_accumulation_steps': 1,
    'learning_rate': 3e-4,
    'use_fp16': True,                # Mixed precision training
    'apply_quantization': False,     # Không quantize khi training
    'save_steps': 500,               # Lưu checkpoint mỗi 500 steps
    'eval_steps': 500,               # Evaluate mỗi 500 steps
}

# Output directories
OUTPUT_DIR = Path(f"{DRIVE_ROOT}/models/wav2vec2-vietnamese")
FINAL_MODEL_DIR = Path(f"{DRIVE_ROOT}/final_model")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FINAL_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Save config
with open(OUTPUT_DIR / 'config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Configuration:")
for key, value in config.items():
    print(f"   - {key}: {value}")
print(f"\n📁 Output: {OUTPUT_DIR}")

## 7️⃣ Load Processor & Datasets

In [ ]:
from transformers import Wav2Vec2Processor
from src.data.preprocessing import load_and_prepare_datasets

print("Loading processor...")
processor = Wav2Vec2Processor.from_pretrained(config['pretrained_model'])

print("\nLoading datasets...")
print("⚠️ This may take 5-10 minutes...")

# Change to Vietnamese-asr directory to use relative paths
os.chdir('/content/Vietnamese-asr')
print(f"Working directory: {os.getcwd()}")

train_dataset, val_dataset, test_dataset = load_and_prepare_datasets(
    str(DATA_DIR / 'train.jsonl'),
    str(DATA_DIR / 'validation.jsonl'),
    str(DATA_DIR / 'test.jsonl'),
    processor
)

print(f"\n✅ Datasets loaded:")
print(f"   - Train: {len(train_dataset):,} samples")
print(f"   - Validation: {len(val_dataset):,} samples")
print(f"   - Test: {len(test_dataset):,} samples")

## 8️⃣ Create Model

In [ ]:
from src.training.train_wav2vec2 import create_model

print("Creating model...")
vocab_size = len(processor.tokenizer)
model = create_model(vocab_size, config['pretrained_model'])

# Move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✅ Model ready on {device}")
print(f"   - Total parameters: {total_params:,}")
print(f"   - Trainable: {trainable_params:,}")
print(f"   - Frozen: {total_params - trainable_params:,}")

## 9️⃣ Start Training

**⏱️ Estimated time: 15-20 hours on T4 GPU**

In [ ]:
from src.training.train_wav2vec2 import train_model

print("="*60)
print("🚀 Starting Training...")
print("="*60)
print("\n⚠️ IMPORTANT:")
print("   - Keep this tab open!")
print("   - Colab timeout: ~12 hours")
print("   - Checkpoints auto-saved to Drive every 500 steps")
print("\n" + "="*60 + "\n")

# Train
trainer = train_model(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    processor=processor,
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=config['num_train_epochs'],
    batch_size=config['batch_size'],
    gradient_accumulation_steps=config['gradient_accumulation_steps'],
    learning_rate=config['learning_rate'],
    use_fp16=config['use_fp16']
)

## 🔟 Save Final Model

In [ ]:
print("Saving final model...")

# Save model
trainer.save_model(str(FINAL_MODEL_DIR))
processor.save_pretrained(str(FINAL_MODEL_DIR))

# Save training history
import pandas as pd
if hasattr(trainer.state, 'log_history'):
    history_df = pd.DataFrame(trainer.state.log_history)
    history_df.to_csv(f"{DRIVE_ROOT}/training_history.csv", index=False)
    print(f"✓ Training history saved")

print(f"\n✅ Training completed!")
print(f"📦 Final model: {FINAL_MODEL_DIR}")
print(f"\n💡 Model đã lưu vào Google Drive, bạn có thể:")
print(f"   1. Download về máy từ Drive")
print(f"   2. Dùng trực tiếp từ Drive trong notebook khác")
print(f"   3. Upload lên HuggingFace Hub")

## 📊 Monitor Training (Optional)

Chạy cell này trong lúc training để theo dõi

In [ ]:
# TensorBoard
%load_ext tensorboard
%tensorboard --logdir {OUTPUT_DIR}/runs

In [ ]:
# GPU monitoring
!nvidia-smi

In [ ]:
# Check latest checkpoint
!ls -lh {OUTPUT_DIR}/checkpoint-*/ | tail -5

## 🎯 Test Model (After Training)

In [ ]:
# Load trained model
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf

model = Wav2Vec2ForCTC.from_pretrained(str(FINAL_MODEL_DIR))
processor = Wav2Vec2Processor.from_pretrained(str(FINAL_MODEL_DIR))
model = model.to(device)
model.eval()

print("✅ Model loaded for inference")

In [ ]:
# Transcribe audio file
def transcribe(audio_path):
    # Load audio
    speech, sr = sf.read(audio_path)
    
    # Process
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Decode
    pred_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(pred_ids)[0]
    
    return transcription

# Test
# audio_file = "/path/to/audio.wav"
# result = transcribe(audio_file)
# print(f"Transcription: {result}")

---

## 📌 Tips

### Tránh Colab timeout:
- Training mất ~15-20h, Colab free timeout sau ~12h
- **Giải pháp:** Chia nhỏ training thành nhiều session
  ```python
  # Session 1: Train 10 epochs
  config['num_train_epochs'] = 10
  
  # Session 2: Resume từ checkpoint, train thêm 10 epochs
  config['resume_from_checkpoint'] = str(OUTPUT_DIR / 'checkpoint-5000')
  config['num_train_epochs'] = 20
  ```

### Colab Pro:
- Timeout: ~24h
- Better GPU: A100/V100
- Training time: ~8-10h

### Auto-save to Drive:
Checkpoints tự động lưu vào Drive mỗi 500 steps, an toàn nếu Colab disconnect!

---